In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from time import time

In [2]:
with open("userwiseReadHistory.pkl", 'rb') as f:
    userwiseReadHistory = pickle.load(f)

In [3]:
with open("metaDataForRecomm.pkl", 'rb') as f:
    metaDataForRecomm = pickle.load(f)

In [4]:
with open('userFollowers.pkl', 'rb') as f:
    userFollowers = pickle.load(f)

In [84]:
with open("testUsersDict.pkl", 'rb') as f:
    testUsersDict = pickle.load(f)

In [6]:
with open("popularityOrdered.pkl", 'rb') as f:
    popularityOrdered = pickle.load(f)

In [7]:
with open("titleDict.pkl", 'rb') as f:
    titleDict = pickle.load(f)

In [8]:
with open("titleTfidfMatrix.pkl", 'rb') as f:
    titleTfidfMatrix = pickle.load(f)

In [9]:
with open("predict/test.users", 'r') as f:
    testUsers = f.read().strip().split()

In [10]:
followings = userFollowers[testUsers[5]]
followingDict = dict(zip(followings, np.zeros(len(followings))))

In [41]:
metaDataForRecomm.shape

(629252, 3)

In [11]:
popularityArticles = [x[0] for x in popularityOrdered]
popularityRankDict = {k:v for v, k in enumerate(popularityArticles)}

In [95]:
titleTfidfMatrix

<643104x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 3435023 stored elements in Compressed Sparse Row format>

In [98]:
metaDataForRecomm.tail()

,articleId,datetime,writer
643098,@nsync1021_14,2019-01-13 12:57:57,@nsync1021
643100,@reading15m_575,2018-10-31 20:14:39,@reading15m
643101,@hje3884_118,2017-11-06 17:36:38,@hje3884
643102,@julieleekgep_12,2018-10-31 22:49:16,@julieleekgep
643103,@julieleekgep_13,2018-11-01 03:24:46,@julieleekgep


In [104]:
testUsersDict[testUsers[0]][0]

0.18691588785046728

In [33]:
recommRes = {}
detailForTestUsers = {}

In [13]:
maxFollowRate = 0.9
noveltyFullScore = 30
othersFullScore = 100 - noveltyFullScore

In [137]:
res = open("saveRes.txt", 'w')

In [35]:
for count, user in enumerate(testUsers[3250:3500]) : 
    
    print(count+3250, user)
    
    try : 
        history = userwiseReadHistory[user]
    except KeyError : 
        history = []
    try : 
        followings = userFollowers[user]
    except KeyError : 
        followings = []
        
    followRate = testUsersDict[user][0]
    readerVector = testUsersDict[user][-1]
    
#     print(history)
#     print(followings)
    
    print("followRate : %f" % followRate)
    
    recommRes[user] = []
    detailForTestUsers[user] = []
    
    if len(history)> 0 :
        historyIndices = [titleDict[x] if x in titleDict else -1 for x in history]
    
#         print(historyIndices)
        
        alreadyReadUniqueIndices = np.unique(historyIndices)
        alreadyReadUniqueIndices = alreadyReadUniqueIndices[np.where(alreadyReadUniqueIndices>=0)]
    
#         print(alreadyReadUniqueIndices)
        
        unreadArticles = metaDataForRecomm.drop(alreadyReadUniqueIndices)
        
        print("history exists; cosine similarity calculation")
        coss = titleTfidfMatrix.dot(readerVector.transpose()).toarray()
        
    else : 
        unreadArticles = metaDataForRecomm
    
    if len(followings) > 0 : 
        followingArticleFilter = np.isin(unreadArticles.writer, followings)
        unreadArticlesFromFollowings = unreadArticles[followingArticleFilter]
#         print(unreadArticlesFromFollowings.head(10))
        unreadArticlesFromOthers = unreadArticles[~followingArticleFilter]
        
        if unreadArticlesFromFollowings.shape[0] :
            articleDfs = [unreadArticlesFromFollowings, unreadArticlesFromOthers]
        else : 
            articleDfs = [unreadArticlesFromOthers]
    else : 
        articleDfs = [unreadArticles]
    
    # dealing with a case that following articles are less than necessary
    howManyFromFollowings = min(unreadArticlesFromFollowings.shape[0], round(followRate * 100), int(maxFollowRate*100))
    howManyFromOthers = 100-howManyFromFollowings
    
    howMany = [howManyFromFollowings, howManyFromOthers]
    
    similarityFullScore = min(round(othersFullScore * followRate), othersFullScore*maxFollowRate)
    popularityFullScore = othersFullScore - similarityFullScore
    
    print(howManyFromFollowings, howManyFromOthers, similarityFullScore, popularityFullScore)
    
    print("start calculation")
    
    for idx, articleDf in enumerate(articleDfs) : 
        print("articleDf %d" % idx)
        totalArticles = articleDf.shape[0]
#         print('totalNum %d' % totalArticles)
        articlesNumpy = articleDf.articleId.values
        classType = ""
        if isinstance(articleDfs[0], int) : 
            classType = 1
        else : 
            if idx == 0 :
                classType = 0
            else : 
                classType = 1
        
        scoreDict = dict(zip(articlesNumpy, [[classType, 0, 0, 0, 0] for x in range(totalArticles)]))
        
        #calculating novelty score
        print("novelty score")
        noveltySortedArticles = articleDf.sort_values('datetime').articleId
        scoreList = np.linspace(0, noveltyFullScore, totalArticles)
        
        for count, article in enumerate(noveltySortedArticles) : 
            scoreDict[article][1] = scoreList[count]
            
        #calculating similarity score only when history exists
        print("similarity score")
        if len(history) > 0 : 
#             print("matrix slicing")
#             slicedTfidfMatrix = titleTfidfMatrix[articleDf.index] # filter necessary vectors
            print('slicing coss')
            cossSliced = coss[articleDf.index]
#             print(cossSliced)
            print('sorting')
            similarSorted = sorted(zip(articlesNumpy, cossSliced), key = lambda x : x[1][0])
            scoreList = np.linspace(0, similarityFullScore, totalArticles)
            
            print('giving similarity score')
            for count, articleAndScore in enumerate(similarSorted) : 
#                 print(articleAndScore)
                scoreDict[articleAndScore[0]][2] = scoreList[count]

#         print(scoreDict)
        
        # popularity score
        print("popularity score")
        meanValue = popularityFullScore / 2
        
        popInfoExists = articleDf[[True if x in popularityRankDict else False for x in articlesNumpy]]
        popInfoNon = articleDf.drop(popInfoExists.index)
        
#         print(popInfoNon.head(10))
        print('sorting popularity')
        popInfoExistsSorted = sorted(popInfoExists.articleId, key=lambda x : popularityRankDict.get(x))
#         print(popInfoExistsSorted[:10])
        
        scoreList = np.linspace(0, popularityFullScore, len(popInfoExistsSorted))
        
        for count, article in enumerate(popInfoExistsSorted) : 
            scoreDict[article][3] = scoreList[count]
            
        for article in popInfoNon.articleId : 
            scoreDict[article][3] = meanValue
            
        # summing all scores
        print('summing')
        totalSorted = []
        for article in scoreDict : 
            scoreDict[article][4] = sum(scoreDict[article][1:4])
        
        totalSorted = sorted(scoreDict.items(), key = lambda x : x[1][4], reverse=True)
        
#         print(totalSorted[:5])
        
        print("assigning results")
#         print(howMany[idx])
        recommRes[user] += [x[0] for x in totalSorted[:howMany[idx]]]
        detailForTestUsers[user] += totalSorted[:howMany[idx]]

3250 #00f01d1d787d5288797ba2322a5c59a1
followRate : 0.400000
history exists; cosine similarity calculation
40 60 28 42
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3251 #865137a11ebaa067eabc83b4d8d7a96f
followRate : 0.743392
history exists; cosine similarity calculation
74 26 52 18
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3252 #37659f3d709409f3aa9fba71303f48c3
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start ca

giving similarity score
popularity score
sorting popularity
summing
assigning results
3269 #097c1c194f69497fc26bfd439c084615
followRate : 0.728111
history exists; cosine similarity calculation
73 27 51 19
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3270 #b5f0c94fbb2a2fd7381a247c8dc8599f
followRate : 0.026087
history exists; cosine similarity calculation
3 97 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3271 #6888e3a79290f4b66c17155af463127f
fol

summing
assigning results
3288 #5f2f98d3d25da45b4c39b568a5416364
followRate : 0.258824
history exists; cosine similarity calculation
26 74 18 52
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3289 #5b0c99363edfed2d8ee51f45250ff849
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3290 #4863e55f85e7635d925f591e82987bc7
followRate : 0.507576
history exists; cosine similarity calcul

similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3307 #3c542092de96effe7b27157c55b55351
followRate : 0.033296
history exists; cosine similarity calculation
3 97 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3308 #bad980e76ce1ca668a57a02fe1b973fd
followRate : 0.095571
history exists; cosine similarity calculation
10 90 7 63
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3309 

assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3326 #990f01bb93545c4651cffd54f3b7f3e6
followRate : 0.303665
history exists; cosine similarity calculation
30 70 21 49
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3327 #f1429116afaf11ff5d4c30f86cf6694b
followRate : 0.583548
history exists; cosine similarity calculation
58 42 41 29
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sor

...

34 66 24 46
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3421 #1a32c64bbdd9bb6ed1f62b5a00d4950c
followRate : 0.444130
history exists; cosine similarity calculation
44 56 31 39
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3422 #5090dd88b6d3f8f754b608c85bf7d432
followRate : 0.155963
history exists; cosine similarity calculation
16 84 11 59
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
populari

87 13 61 9
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3440 #c65d7bc176069f0d92104fa690407f49
followRate : 0.714286
history exists; cosine similarity calculation
71 29 50 20
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3441 #f737a56b755a8a79747646a2b632da49
followRate : 0.213592
history exists; cosine similarity calculation
21 79 15 55
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularit

53 47 37 33
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3459 #7ba5ff376c52b00929c762d616636a62
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3460 #2b4ef54663bfba74b7645ed6d1c58e2b
followRate : 0.657143
history exists; cosine similarity calculation
66 34 46 24
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularit

novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3478 #1bb3bb8ab9302057fc50a3c101d9378e
followRate : 0.157282
history exists; cosine similarity calculation
16 84 11 59
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3479 #e3a9adb0720800c61b851fd38d189c1a
followRate : 0.035000
history exists; cosine similarity calculation
4 96 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assignin

2 98 1 69
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3497 #d66560626fa204868dacb73ed71f1415
followRate : 0.430149
history exists; cosine similarity calculation
43 57 30 40
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3498 #2848ea8028085a8c87416f6048029e95
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity 

In [83]:
testUsersDict["#28f6ed60737aafe587e3ec4eb53b23db"]

[0.02702702702702703,
 3,
 111,
 <1x60000 sparse matrix of type '<class 'numpy.float64'>'
 	with 441 stored elements in Compressed Sparse Row format>]

In [85]:
for count, user in enumerate(remaining2) : 
    
    print(count, user)
    
    try : 
        history = userwiseReadHistory[user]
    except KeyError : 
        history = []
    try : 
        followings = userFollowers[user]
    except KeyError : 
        followings = []
        
    followRate = testUsersDict[user][0]
    readerVector = testUsersDict[user][-1]
    
#     print(history)
#     print(followings)
    
    print("followRate : %f" % followRate)
    
    recommRes[user] = []
    detailForTestUsers[user] = []
    
    if len(history)> 0 :
        historyIndices = [titleDict[x] if x in titleDict else -1 for x in history]
    
#         print(historyIndices)
        
        alreadyReadUniqueIndices = np.unique(historyIndices)
        alreadyReadUniqueIndices = alreadyReadUniqueIndices[np.where(alreadyReadUniqueIndices>=0)]
    
#         print(alreadyReadUniqueIndices)
        
        unreadArticles = metaDataForRecomm.drop(alreadyReadUniqueIndices)
        
        print("history exists; cosine similarity calculation")
        coss = titleTfidfMatrix.dot(readerVector.transpose()).toarray()
        
    else : 
        unreadArticles = metaDataForRecomm
#     print(followings)
    if len(followings) > 0 : 
        followingArticleFilter = np.isin(unreadArticles.writer, followings)
        unreadArticlesFromFollowings = unreadArticles[followingArticleFilter]
#         print(unreadArticlesFromFollowings.head(10))
        unreadArticlesFromOthers = unreadArticles[~followingArticleFilter]
        
        print(unreadArticlesFromFollowings.shape)
        if unreadArticlesFromFollowings.shape[0] :
            articleDfs = [unreadArticlesFromFollowings, unreadArticlesFromOthers]
        else : 
            articleDfs = [unreadArticlesFromOthers]
    else : 
        articleDfs = [unreadArticles]
    
#     print(articleDfs)
    # dealing with a case that following articles are less than necessary
    howManyFromFollowings = min(unreadArticlesFromFollowings.shape[0], round(followRate * 100), int(maxFollowRate*100))
    howManyFromOthers = 100-howManyFromFollowings
    
#     print(unreadArticlesFromFollowings.shape)
    howMany = [howManyFromFollowings, howManyFromOthers]
    
    similarityFullScore = min(round(othersFullScore * followRate), othersFullScore*maxFollowRate)
    popularityFullScore = othersFullScore - similarityFullScore
    
    print(howManyFromFollowings, howManyFromOthers, similarityFullScore, popularityFullScore)
    
    print("start calculation")
    
    for idx, articleDf in enumerate(articleDfs) : 
        print("articleDf %d" % idx)
        totalArticles = articleDf.shape[0]
        print('totalNum %d' % totalArticles)
        articlesNumpy = articleDf.articleId.values
        classType = ""
        if len(articleDfs) == 1 : 
            idx = 1
            classType = 1
        else : 
            if idx == 0 :
                classType = 0
            else : 
                classType = 1
        
        scoreDict = dict(zip(articlesNumpy, [[classType, 0, 0, 0, 0] for x in range(totalArticles)]))
        
        #calculating novelty score
        print("novelty score")
        noveltySortedArticles = articleDf.sort_values('datetime').articleId
        scoreList = np.linspace(0, noveltyFullScore, totalArticles)
        
        for count, article in enumerate(noveltySortedArticles) : 
            scoreDict[article][1] = scoreList[count]
            
        #calculating similarity score only when history exists
        print("similarity score")
        if len(history) > 0 : 
#             print("matrix slicing")
#             slicedTfidfMatrix = titleTfidfMatrix[articleDf.index] # filter necessary vectors
            print('slicing coss')
            cossSliced = coss[articleDf.index]
#             print(cossSliced)
            print('sorting')
            similarSorted = sorted(zip(articlesNumpy, cossSliced), key = lambda x : x[1][0])
            scoreList = np.linspace(0, similarityFullScore, totalArticles)
            
            print('giving similarity score')
            for count, articleAndScore in enumerate(similarSorted) : 
#                 print(articleAndScore)
                scoreDict[articleAndScore[0]][2] = scoreList[count]

#         print(scoreDict)
        
        # popularity score
        print("popularity score")
        meanValue = popularityFullScore / 2
        
        popInfoExists = articleDf[[True if x in popularityRankDict else False for x in articlesNumpy]]
        popInfoNon = articleDf.drop(popInfoExists.index)
        
#         print(popInfoNon.head(10))
        print('sorting popularity')
        popInfoExistsSorted = sorted(popInfoExists.articleId, key=lambda x : popularityRankDict.get(x))
#         print(popInfoExistsSorted[:10])
        
        scoreList = np.linspace(0, popularityFullScore, len(popInfoExistsSorted))
        
        for count, article in enumerate(popInfoExistsSorted) : 
            scoreDict[article][3] = scoreList[count]
            
        for article in popInfoNon.articleId : 
            scoreDict[article][3] = meanValue
            
        # summing all scores
        print('summing')
        totalSorted = []
        for article in scoreDict : 
            scoreDict[article][4] = sum(scoreDict[article][1:4])
        
        print(len(scoreDict))
        
        totalSorted = sorted(scoreDict.items(), key = lambda x : x[1][4], reverse=True)
        
        print(totalSorted[:5])
        
        print("assigning results")
        print(howMany[idx])
        recommRes[user] += [x[0] for x in totalSorted[:howMany[idx]]]
        detailForTestUsers[user] += totalSorted[:howMany[idx]]

0 #28f6ed60737aafe587e3ec4eb53b23db
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
totalNum 629169
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
629169
[('@aksen1668_21', [1, 25.167951961956106, 0.0, 69.97926744989634, 95.14721941185245]), ('@inihiai_54', [1, 25.149117564783968, 0.0, 69.91368789424929, 95.06280545903326]), ('@studioyeun_28', [1, 25.091136230704677, 0.0, 69.80032054581011, 94.89145677651479]), ('@badventure_391', [1, 25.076593215166696, 0.0, 69.77517681483333, 94.85177003000003]), ('@badventure_421', [1, 25.07974022836508, 0.0, 69.75503242217941, 94.8347726505445])]
assigning results
100
1 #d2d09d0a95c6929604fe9d2501e441b1
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
totalNum 629244
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score

In [79]:
userFollowers[remaining2[0]]

KeyError: '#28f6ed60737aafe587e3ec4eb53b23db'

In [72]:
len(recommRes[remaining2[0]])

97

In [140]:
with open("recommRes3.pkl", 'wb') as f:
    pickle.dump(recommRes, f)

with open('detailForTestUsers3.pkl', 'wb') as f:
    pickle.dump(detailForTestUsers, f)

In [81]:
for i in zip(titleTfidfMatrix[articleDf.index].dot(readerVector.transpose()).toarray(), range(665)):
    print(i[0][0])
    break

0.0


In [114]:
len(list(recommRes.values())[0])

38

In [72]:
readerVector

<1x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 342 stored elements in Compressed Sparse Row format>

In [48]:
np.linspace(0, 9, 10)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [55]:
g= [3,4,5,2,1]

In [56]:
g.sort()

In [76]:
for i in zip(g, ['a', 'b', 'c', 'd', 'e']):
    print (i)

(1, 'a')
(2, 'b')
(3, 'c')
(4, 'd')
(5, 'e')


In [100]:
popularityRankDict

{'@brunch_151': 0,
 '@sweetannie_145': 1,
 '@intlovesong_28': 2,
 '@tenbody_1305': 3,
 '@conbus_43': 4,
 '@chofang1_15': 5,
 '@seochogirl_1': 6,
 '@brunch_145': 7,
 '@seochogirl_18': 8,
 '@deckey1985_51': 9,
 '@dailylife_207': 10,
 '@steven_179': 11,
 '@brunch_152': 12,
 '@seochogirl_17': 13,
 '@seochogirl_16': 14,
 '@tenbody_1164': 15,
 '@seochogirl_6': 16,
 '@brunch_149': 17,
 '@hyehyodam_19': 18,
 '@hjl0520_26': 19,
 '@shanghaiesther_46': 20,
 '@seochogirl_7': 21,
 '@seochogirl_8': 22,
 '@noey_130': 23,
 '@brunch_148': 24,
 '@seochogirl_11': 25,
 '@seochogirl_28': 26,
 '@wootaiyoung_85': 27,
 '@seochogirl_3': 28,
 '@seochogirl_12': 29,
 '@bzup_281': 30,
 '@seochogirl_2': 31,
 '@seochogirl_20': 32,
 '@seochogirl_29': 33,
 '@seochogirl_10': 34,
 '@kam_60': 35,
 '@seochogirl_13': 36,
 '@seochogirl_14': 37,
 '@brunch_141': 38,
 '@seochogirl_5': 39,
 '@seochogirl_9': 40,
 '@seochogirl_15': 41,
 '@mothertive_66': 42,
 '@kotatsudiary_66': 43,
 '@wikitree_54': 44,
 '@dailylife_219': 45,
 '@

In [122]:
len(recommRes[testUsers[0]])

100

In [132]:
del recommRes[testUsers[69]]

In [136]:
del detailForTestUsers[testUsers[69]]

In [91]:
with open("predict/dev.users", 'r') as f:
    devUsers = f.read().strip().split()

In [92]:
res1 = open('devuser/recommend.txt', 'w')
res2 = open('testuser/recommend.txt', 'w')

for user in testUsers : 
    eachResText = " ".join(totalRecomm[user])
    eachResText = user + " " + eachResText + '\n'
    
    res2.write(eachResText)
    if user in devUsers : 
        res1.write(eachResText)
        
res2.close()
res1.close()

In [15]:
with open('recommRes.pkl', 'rb') as f:
    recommRes = pickle.load(f)
    
with open('recommRes3.pkl', 'rb') as f:
    recommRes3 = pickle.load(f)
    
with open('recommRes4.pkl', 'rb') as f:
    recommRes4 = pickle.load(f)
    
with open('recommRes6.pkl', 'rb') as f:
    recommRes6 = pickle.load(f)

In [16]:
with open('recommRes2.pkl', 'rb') as f:
    recommRes2 = pickle.load(f)

In [19]:
with open("saveRes.txt", 'r') as f:
    saveRes = f.read()

In [20]:
saveRes = saveRes.strip().split()

In [22]:
recommRes13 = {}
for line in saveRes : 
    recommRes13[line[0]] = line[1:]

In [17]:
totalRecomm = {}

In [86]:
for i in recommRes:
    totalRecomm[i] = recommRes[i]

In [87]:
len(totalRecomm)

5000

In [32]:
with open('totalRecomm.pkl', 'wb') as f:
    pickle.dump(totalRecomm, f)

In [42]:
save = []
for i in list(totalRecomm.keys()):
    if i not in testUsers: 
        save.append(i)

In [45]:
del totalRecomm["@"]

In [88]:
sum([len(x) for x in totalRecomm.values()])

500000

In [69]:
save = []
remaining2 = []
for count, user in enumerate(testUsers) : 
    temp = totalRecomm[user]
    if len(temp) != 100:
        remaining2.append(user)
        save.append(count)

In [66]:
save

[579, 3280, 4426, 4631, 4689]

In [68]:
len(recommRes[testUsers[save[0]]])

97

In [65]:
len(save)

5

In [51]:
remainings = [testUsers[x] for x in save]

In [52]:
remainings

['#7f0937371581b66144e680f5fcf54bfc',
 '#163080b31c2b37253ec19b2602bfd5b9',
 '#f7d88470c7bd36a958fd85e76f73656c',
 '#d28f6759946781c03fdc6aa0a8ae4c61',
 '#99bc118126fc4173762d6ba6b005e931',
 '#28f6ed60737aafe587e3ec4eb53b23db',
 '#1f069f17a14280cadca9e11e2fb9de7a',
 '#ee15b7207a931465f27df829e10628ac',
 '#f6317132d17cbc7f365514118ab68ec5',
 '#155a8f6cdcec512e7d8efa3671dcf90b',
 '#4dfad4d56fa3863b75e8907ff5f8a149',
 '#a2f3379335e7d3e97588180860bbebfb',
 '#16e9f7b4752fccffe52e92b3f9b90586',
 '#99e850fee0850910e049143cae1f7386',
 '#5118fd467fc5191243991fb324eaa4a0',
 '#6b3f9ae509f7ce1925b7db79cdc50bfb',
 '#270f067fec220d7ab9e36ba69f8b94d4',
 '#0a92e1983b1ec8e55f4f64ddac118f4c',
 '#1fbaa509fa881ef2e8b4c356c163c242',
 '#92e7d9d18cf6b67cf17a9e74fb336d2a',
 '#290c6077def84b40626fd4f15e83a602',
 '#d491ee7aa6ebd665bee51d559ae46345',
 '#c1bc1733826b6bc5935f779d2fa21e7e',
 '#49a30f378f28d7445f5c3a1862945489',
 '#053e31c7b0443fc51b2dad5703fa43b8',
 '#0a05155f0839b072c716360c8a3d26b7',
 '#e22dfaf2e